In [19]:
# Import dependencies and set up TTL. This is being done here unlike the other modules because
# the Tensorflow classifier needs 0 - numclasses - 1 instead of 1-5 so its slightly different.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization

# Load and preprocess your CSV
df = pd.read_csv("../datasets/allsymptomsdisease.csv")
df = df.dropna()
df['text'] = df['text'].astype(str)
df['label'] = df['label'].astype(float)

# Bin labels (and subtract 1 for 0-based indexing)
def binner(scores):
    output = []
    for score in scores:
        if score > 900:
            output.append(1)
        elif score > 650:
            output.append(2)
        elif score > 400:
            output.append(3)
        elif score > 200:
            output.append(4)
        else:
            output.append(5)
    return output

df['label'] = binner(df['label'].tolist())
df['label'] = df['label'] - 1

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['text']).toarray()
y = np.array(df['label'])

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Define model builder for Keras Tuner

def model_builder(hp):
    model = Sequential()

    # Input Layer
    model.add(Dense(
        units=hp.Int('input_units', 128, 512, step=64),
        activation=hp.Choice('input_activation', ['relu', 'tanh']),
        input_shape=(X_train.shape[1],),
        kernel_regularizer=regularizers.l2(hp.Float('l2_input', 1e-5, 1e-2, sampling='log'))
    ))
    if hp.Boolean('batchnorm_input'):
        model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_input', 0.1, 0.5, step=0.1)))

    # Hidden Layers (1 to 5)
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', 64, 512, step=64),
            activation=hp.Choice(f'activation_{i}', ['relu', 'tanh']),
            kernel_regularizer=regularizers.l2(hp.Float(f'l2_{i}', 1e-5, 1e-2, sampling='log'))
        ))
        if hp.Boolean(f'batchnorm_{i}'):
            model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f'dropout_{i}', 0.1, 0.6, step=0.1)))

    # Output Layer
    model.add(Dense(5, activation='softmax'))

    # Compile with optimizer and learning rate
    optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])
    learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')

    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    hp.Int('batch_size', 32, 128, step=32)

    return model

# Set up Keras Tuner
tuner = kt.RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=50,
    executions_per_trial=1,
    directory='hp_tuning',
    project_name='severity_classifier'
)

# Add early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Search for best model
tuner.search(X_train, y_train,
             epochs=40,
             validation_split=0.2,
             callbacks=[early_stop],
             verbose=2)




Trial 50 Complete [00h 00m 20s]
val_accuracy: 0.8703879117965698

Best val_accuracy So Far: 0.8798485994338989
Total elapsed time: 00h 21m 45s


In [20]:
# 1. Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters(1)[0]


# 2. Rebuild the model using those hyperparameters
best_model = tuner.hypermodel.build(best_hp)

# 3. Refit model with optimal batch size
best_model.fit(
    X_train,
    y_train,
    epochs=40,
    batch_size=best_hp.get('batch_size'),
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=2
)

# 4. Evaluate on the test set
test_loss, test_acc = best_model.evaluate(X_test, y_test)
y_pred = np.argmax(best_model.predict(X_test), axis=1)

print("Test Accuracy:", test_acc)
print(classification_report(y_test, y_pred, digits=3))

Epoch 1/40
34/34 - 3s - 93ms/step - accuracy: 0.5493 - loss: 3.2047 - val_accuracy: 0.5345 - val_loss: 3.2749
Epoch 2/40
34/34 - 0s - 13ms/step - accuracy: 0.7690 - loss: 2.4879 - val_accuracy: 0.6074 - val_loss: 3.1317
Epoch 3/40
34/34 - 0s - 13ms/step - accuracy: 0.8142 - loss: 2.2829 - val_accuracy: 0.7162 - val_loss: 3.0002
Epoch 4/40
34/34 - 0s - 13ms/step - accuracy: 0.8445 - loss: 2.1088 - val_accuracy: 0.7938 - val_loss: 2.8455
Epoch 5/40
34/34 - 0s - 12ms/step - accuracy: 0.8693 - loss: 1.9945 - val_accuracy: 0.8146 - val_loss: 2.6668
Epoch 6/40
34/34 - 0s - 12ms/step - accuracy: 0.8911 - loss: 1.8934 - val_accuracy: 0.8288 - val_loss: 2.4780
Epoch 7/40
34/34 - 0s - 12ms/step - accuracy: 0.9013 - loss: 1.8114 - val_accuracy: 0.8259 - val_loss: 2.2983
Epoch 8/40
34/34 - 0s - 12ms/step - accuracy: 0.9280 - loss: 1.7074 - val_accuracy: 0.8382 - val_loss: 2.1220
Epoch 9/40
34/34 - 0s - 12ms/step - accuracy: 0.9359 - loss: 1.6397 - val_accuracy: 0.8382 - val_loss: 1.9803
Epoch 10/4

In [16]:
print(best_hp.values)


{'input_units': 192, 'input_activation': 'tanh', 'l2_input': 0.00021777965369913825, 'batchnorm_input': False, 'dropout_input': 0.2, 'num_layers': 3, 'units_0': 256, 'activation_0': 'tanh', 'l2_0': 0.00427775755932623, 'batchnorm_0': True, 'dropout_0': 0.2, 'units_1': 64, 'activation_1': 'tanh', 'l2_1': 0.0005463063812883169, 'batchnorm_1': True, 'dropout_1': 0.1, 'optimizer': 'adam', 'learning_rate': 0.001677109135321793, 'batch_size': 96, 'units_2': 384, 'activation_2': 'tanh', 'l2_2': 0.0008854322452566148, 'batchnorm_2': False, 'dropout_2': 0.2, 'units_3': 256, 'activation_3': 'relu', 'l2_3': 0.0002851509951970098, 'batchnorm_3': False, 'dropout_3': 0.4, 'units_4': 128, 'activation_4': 'relu', 'l2_4': 0.00011908740025576155, 'batchnorm_4': True, 'dropout_4': 0.5}
